In [1]:
# Text Similarity-Modelling

# XG 부스트 모델(데이터를 TF-IDF 과정을 거쳐서 학습)

# 모델 구현
import numpy as np
np_load_old = np.load  # 전처리한 데이터 가져오기에서 np.load 실행시 오류가 있어서 수정
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
import os

import json

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_CLEAN_Q1_DATA = 'train_clean_q1.csv'
TRAIN_CLEAN_Q2_DATA = 'train_clean_q2.csv'
# TRAIN_LABEL_DATA_FILE = 'train_label.npy'

#훈련 데이터 가져오기(문자열 데이터)
train_q1_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_Q1_DATA )
train_q2_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_Q2_DATA )
# train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))
train_q1_data.head()

# train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

,questions1,is_duplicate
0,how do you cook canned kidney beans,0
1,how can i get the guy i like to like me again,0
2,how can 46x30 mm rounds have more armor pierci...,0
3,whats the difference in meaning between these ...,0
4,is wikibuycom legit or a scam,0


ValueError: Object arrays cannot be loaded when allow_pickle=False
 

코드를 돌리던 중, 결과를 확인하려고 np.load를 이용했지만, 갑자기 위의 오류가 뜨면서 로드되지 않았다.

검색을 통해 찾아보니, 해당 allow_pickle을 True로 바꿔주어 사용하면 오류를 해결할 수 있었다.

import numpy as np

#### 먼저 기존의 np.load를 np_load_old에 저장해둠.
np_load_old = np.load

#### 기존의 parameter을 바꿔줌
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
위의 코드를 이용하여 기존의 np.load를 바꿔주고, 다시 기존의 것을 사용하고 싶다면 다음과 같이 다시 불러오면 된다.

np.load = np_load_old

출처 : https://d-tail.tistory.com/31

In [2]:
train_q1_input = list(train_q1_data['questions1'])
train_q2_input = list( train_q2_data['questions2'])
train_labels = list(train_q1_data['is_duplicate'])
# print(train_q1_input[0:42])



In [3]:
# train_data_df = pd.DataFrame({'questions1': train_q1_input, 'questions2': train_q2_input, 'is_duplicate': train_labels})
# df = train_data_df
# df_check = train_data_df.isnull()
# print(df_check)

# check_for_nan = df.isnull().values.any()
# print(check_for_nan) #Nan값이 있다면 True출력

# total_nan_values = df.isnull().sum().sum()
# print(total_nan_values)

In [4]:
# df_dop_row = df.dropna(axis=0)  # Nan 값이 있는 행 제거

# total_nan_values = df_dop_row.isnull().sum().sum()
# print(total_nan_values) #Nan 값이 몇개 있는지 출력

In [5]:
# train_q1_input = list(df_dop_row['questions1'])
# train_q2_input = list(df_dop_row['questions2'])
# train_labels = list(df_dop_row['is_duplicate'])

In [6]:
# vectorizer = TfidfVectorizer() 
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=31) 
train_q1_data = vectorizer.fit_transform(train_q1_input).toarray()
train_q2_data = vectorizer.fit_transform(train_q2_input).toarray()
train_labels = np.array(train_labels)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
train_q1_data
# print(train_q1_data.shape)

In [ ]:
print(train_q1_data[0])

In [ ]:
print(train_q2_data[0:2])

In [ ]:
# 두 질문을 하나의 쌍으로
train_input = np.stack((train_q1_data, train_q2_data), axis=1)
# print(train_input)

In [ ]:
# 학습 데이터를 훈련 데이터와 평가 데이터로 구분
from sklearn.model_selection import train_test_split
train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)

In [ ]:
# xgboost 모델 형식에 맞게 입력갑 가공
import xgboost as xgb

train_data = xgb.DMatrix(train_input.sum(axis=1), label=train_label)    #학습 데이터 읽어 오기
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label)       #평가 데이터 읽어 오기

data_list = [(train_data, 'train'), (eval_data, 'valid')]

In [ ]:
# 목적함수, 평가지표 설정
params = {}
params['obejective'] = 'binary:logistic'    # 목적함수 : 이진 로지스틱 함수
params['eval_metric'] = 'rmse'  # 평가지표 : root mean square error

bst = xgb.train(params, train_data, num_boost_round=1000, evals = data_list, early_stopping_rounds=10)
# num_boost_round(=epochs)= 1000, early stopping 10에서 조기 멈춤

In [ ]:
# 전처리한 데이터 가져오기
# TEST_Q1_DATA_FILE = 'test_q1.npy'
# TEST_Q2_DATA_FILE = 'test_q2.npy'
# TEST_ID_DATA_FILE = 'test_id.npy'

# test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
# test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
# test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

TEST_CLEAN_Q1_DATA = 'test_clean_q1.csv'
TEST_CLEAN_Q2_DATA = 'test_clean_q2.csv'

test_q1_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_Q1_DATA)
test_q2_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_Q2_DATA)

test_q1_data.head()


In [ ]:
test_q1_input = list(test_q1_data['questions1'])
test_q2_input = list( test_q2_data['questions2'])
test_id_input = list(test_q2_data['test_id'])

In [ ]:
# test_data_df = pd.DataFrame({'questions1': test_q1_input, 'questions2': test_q2_input, 'id': test_id_input})

In [ ]:
# # Nan 있는지 확인
# df = test_data_df
# df_check = test_data_df.isnull()
# print(df_check)

# check_for_nan = df.isnull().values.any()
# print(check_for_nan) #Nan값이 있다면 True출력

# total_nan_values = df.isnull().sum().sum()
# print(total_nan_values)

In [ ]:
# df_dop_row = df.dropna(axis=0)  # Nan 값이 있는 행 제거

# total_nan_values = df_dop_row.isnull().sum().sum()
# print(total_nan_values) #Nan 값이 몇개 있는지 출력

In [ ]:
# test_q1_input = list(df_dop_row['questions1'])
# test_q2_input = list(df_dop_row['questions2'])
# test_id_input = list(df_dop_row['id'])

In [ ]:
# vectorizer = TfidfVectorizer() 
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=31) 
test_q1_data = vectorizer.fit_transform(test_q1_input).toarray()
test_q2_data = vectorizer.fit_transform(test_q2_input).toarray()
test_id_data = np.array(test_id_input)

In [ ]:
# Predict 함수로 예측
test_input = np.stack((test_q1_data, test_q2_data), axis=1)
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

# 결괏값 파일로 저장
DATA_OUT_PATH = './data_out/'
import pandas as pd

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
output.to_csv(DATA_OUT_PATH + 'simple_xgb_TF-IDF.csv', index=False)